In [1]:
# === Librerías principales ===
import pandas as pd
import numpy as np

# === Modelado y preprocesamiento ===
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


In [ ]:
# === Cargar el dataset ===
path = 'D:/TFG/codigo/datos/indicadores-filtrados-TODAS-LIGAS.csv'
df = pd.read_csv(path, sep=",")

# === Eliminar columnas no predictoras ===
df.drop(['jornada', 'liga', 'temporada', 'id_indicadores_equipo_prepartido', 'id_partido'], axis=1, inplace=True)

# === Variables predictoras ===
X = df.drop(['resultado_partido', 'resultado_local', 'resultado_visitante'], axis=1)

# === Variables objetivo ===
y_resultado = df['resultado_partido']           # Clasificación: resultado del partido (1, X, 2)
y_goles_local = df['resultado_local']           # Regresión: goles del equipo local
y_goles_visitante = df['resultado_visitante']   # Regresión: goles del equipo visitante


In [4]:
from sklearn.preprocessing import MinMaxScaler

# === Escalado de los datos ===
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# === División para clasificación (resultado del partido) ===
X_train_cls, X_test_cls, y_train_cls, y_test_cls = train_test_split(
    X_scaled, y_resultado, test_size=0.2, random_state=42
)

# === División para regresión (goles local) ===
X_train_loc, X_test_loc, y_train_loc, y_test_loc = train_test_split(
    X_scaled, y_goles_local, test_size=0.2, random_state=42
)

# === División para regresión (goles visitante) ===
X_train_vis, X_test_vis, y_train_vis, y_test_vis = train_test_split(
    X_scaled, y_goles_visitante, test_size=0.2, random_state=42
)


### Predicciones resultado

In [5]:
from sklearn.neural_network import MLPClassifier

# === Modelo base ===
mlp_cls = MLPClassifier(max_iter=500, random_state=42)

# === Grid de hiperparámetros ===
param_grid_mlp = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50)],
    'activation': ['relu', 'tanh'],
    'alpha': [0.0001, 0.001],  # Regularización L2
    'solver': ['adam']         # Método de optimización más estable
}

# === Búsqueda de hiperparámetros con validación cruzada ===
grid_search_mlp = GridSearchCV(
    estimator=mlp_cls,
    param_grid=param_grid_mlp,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)

# Entrenamiento del grid search
grid_search_mlp.fit(X_train_cls, y_train_cls)

# Mostrar mejores parámetros encontrados
print("Mejores hiperparámetros para MLPClassifier:")
print(grid_search_mlp.best_params_)


Mejores hiperparámetros para MLPClassifier:
{'activation': 'tanh', 'alpha': 0.001, 'hidden_layer_sizes': (100,), 'solver': 'adam'}


In [6]:
# === Obtener el mejor modelo ===
best_mlp_model = grid_search_mlp.best_estimator_

# === Entrenar sobre el conjunto de entrenamiento completo ===
best_mlp_model.fit(X_train_cls, y_train_cls)

# === Predicción sobre el conjunto de prueba ===
y_pred_cls = best_mlp_model.predict(X_test_cls)


In [7]:
# === Evaluación del rendimiento del modelo MLP ===
accuracy = accuracy_score(y_test_cls, y_pred_cls)
print(f"Accuracy del MLPClassifier: {accuracy:.4f}")

# Matriz de confusión
print("Matriz de confusión:")
print(confusion_matrix(y_test_cls, y_pred_cls))

# Informe por clase
print("Informe de clasificación:")
print(classification_report(y_test_cls, y_pred_cls, target_names=["Victoria local", "Empate", "Victoria visitante"]))


Accuracy del MLPClassifier: 0.4811
Matriz de confusión:
[[217  50  10]
 [ 97  68   9]
 [105  44   7]]
Informe de clasificación:
                    precision    recall  f1-score   support

    Victoria local       0.52      0.78      0.62       277
            Empate       0.42      0.39      0.40       174
Victoria visitante       0.27      0.04      0.08       156

          accuracy                           0.48       607
         macro avg       0.40      0.41      0.37       607
      weighted avg       0.43      0.48      0.42       607



### Prediccion equipo local

In [8]:
from sklearn.neural_network import MLPRegressor

# === Modelo base ===
mlp_reg = MLPRegressor(max_iter=500, random_state=42)

# === Grid de hiperparámetros ===
param_grid_reg = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50)],
    'activation': ['relu', 'tanh'],
    'alpha': [0.0001, 0.001],  # Regularización
    'solver': ['adam']
}

# === Búsqueda de hiperparámetros con validación cruzada ===
grid_search_reg = GridSearchCV(
    estimator=mlp_reg,
    param_grid=param_grid_reg,
    cv=5,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1
)

# Ajuste con los datos de goles local
grid_search_reg.fit(X_train_loc, y_train_loc)

# Mostrar mejores parámetros encontrados
print("Mejores hiperparámetros para MLPRegressor (goles local):")
print(grid_search_reg.best_params_)


Mejores hiperparámetros para MLPRegressor (goles local):
{'activation': 'tanh', 'alpha': 0.001, 'hidden_layer_sizes': (100,), 'solver': 'adam'}


In [9]:
# === Modelo final con los mejores parámetros ===
best_mlp_reg_loc = grid_search_reg.best_estimator_

# Entrenamiento sobre todo el conjunto de entrenamiento
best_mlp_reg_loc.fit(X_train_loc, y_train_loc)

# Predicción sobre el conjunto de prueba
y_pred_loc = best_mlp_reg_loc.predict(X_test_loc)


In [10]:
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score

# === RMSE y R² ===
rmse_loc = np.sqrt(mean_squared_error(y_test_loc, y_pred_loc))
r2_loc = r2_score(y_test_loc, y_pred_loc)

print(f"RMSE (goles local): {rmse_loc:.4f}")
print(f"R² Score (goles local): {r2_loc:.4f}")

# === Accuracy redondeado ===
y_pred_loc_round = np.round(y_pred_loc).astype(int)
y_test_loc_round = y_test_loc.astype(int)

acc_loc = accuracy_score(y_test_loc_round, y_pred_loc_round)
print(f"Accuracy exacta (goles locales redondeados): {acc_loc:.4f}")


RMSE (goles local): 1.1946
R² Score (goles local): 0.0349
Accuracy exacta (goles locales redondeados): 0.3410


### Prediccion equipo visitante

In [11]:
# === Modelo base ===
mlp_reg_vis = MLPRegressor(max_iter=500, random_state=42)

# === Grid de hiperparámetros ===
param_grid_reg_vis = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50)],
    'activation': ['relu', 'tanh'],
    'alpha': [0.0001, 0.001],
    'solver': ['adam']
}

# === Búsqueda de hiperparámetros con validación cruzada ===
grid_search_reg_vis = GridSearchCV(
    estimator=mlp_reg_vis,
    param_grid=param_grid_reg_vis,
    cv=5,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1
)

# Ajuste con los datos de goles visitante
grid_search_reg_vis.fit(X_train_vis, y_train_vis)

# Mostrar mejores parámetros encontrados
print("Mejores hiperparámetros para MLPRegressor (goles visitante):")
print(grid_search_reg_vis.best_params_)


Mejores hiperparámetros para MLPRegressor (goles visitante):
{'activation': 'tanh', 'alpha': 0.001, 'hidden_layer_sizes': (50,), 'solver': 'adam'}


In [12]:
# === Entrenar el modelo con los mejores hiperparámetros ===
best_mlp_reg_vis = grid_search_reg_vis.best_estimator_

# Entrenamiento sobre el conjunto de entrenamiento
best_mlp_reg_vis.fit(X_train_vis, y_train_vis)

# Predicción sobre el conjunto de prueba
y_pred_vis = best_mlp_reg_vis.predict(X_test_vis)


In [13]:
# === Cálculo del RMSE y R² ===
rmse_vis = np.sqrt(mean_squared_error(y_test_vis, y_pred_vis))
r2_vis = r2_score(y_test_vis, y_pred_vis)

print(f"RMSE (goles visitante): {rmse_vis:.4f}")
print(f"R² Score (goles visitante): {r2_vis:.4f}")

# === Accuracy redondeado ===
y_pred_vis_round = np.round(y_pred_vis).astype(int)
y_test_vis_round = y_test_vis.astype(int)

acc_vis = accuracy_score(y_test_vis_round, y_pred_vis_round)
print(f"Accuracy exacta (goles visitantes redondeados): {acc_vis:.4f}")


RMSE (goles visitante): 1.1497
R² Score (goles visitante): -0.0100
Accuracy exacta (goles visitantes redondeados): 0.3443
